In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd gdrive/MyDrive/Github/sports-article-classification/
%pip install slackclient
%pip install doccano-client
#%pip install -e .
%pip install wandb
%pip install scikit-multilearn
%pip install scikit-optimize
%cd src/modelling

%cd ../..
%pip install -e .

/content/gdrive/MyDrive/Github/sports-article-classification
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.1/97.1 KB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 23.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 KB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 KB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 KB 5.8 MB/s eta 0:00:00
  Attempting unins

In [3]:
#%cd ../..
#%pip install -e .

In [17]:
from src.data.preprocessing import DataPreprocessorHelland
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
from src.slack_alert.sofus_alert import sofus_alert
import configparser
from src.utils import get_project_root
from src.data.dataset import get_training_data
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, classification_report
from skmultilearn.model_selection import iterative_train_test_split, IterativeStratification
import wandb
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from scipy.sparse import csr_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from skopt.searchcv import BayesSearchCV
import multiprocessing
import time
from skopt.space import Integer
from skopt.space import Real
from skopt.space import Categorical
from skopt.utils import use_named_args


from src.data.nordskog_data import get_data
from src.data.preprocessing import DataPreprocessorNordskog, DataPreprocessorHelland
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from keras.losses import categorical_crossentropy

In [6]:
%cd src/modelling

/content/gdrive/MyDrive/Github/sports-article-classification/src/modelling


In [7]:
data, test_experiment_data = get_data()
data_preprocessor = DataPreprocessorNordskog(data)
data_preprocessor.map_nordskog_data()
data_preprocessor.remove_extra_spaces_from_text()
data = data_preprocessor.data.copy()
label_encoder = LabelEncoder()
data['label'] = label_encoder.fit_transform(data['label'])



test_size = 0.2
X = data['text']
y = data['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=100, shuffle=True, stratify=y)

train_split = pd.concat([X_train, y_train], axis=1)
validation_split = pd.concat([X_test, y_test], axis=1)

vectorizer = TfidfVectorizer()
X_vectorized = vectorizer.fit_transform(train_split['text'])


In [29]:
models = [['RandomForestClassifier', RandomForestClassifier(random_state=100)],
                          ['KNeighborsClassifier', KNeighborsClassifier(metric='euclidean')],
                          ['LinearSVC', LinearSVC(random_state=100)],
                          ['GradientBoostingClassifier', GradientBoostingClassifier(random_state=100)],
                          ['MLPClassifier', MLPClassifier()],
                          ['AdaBoostClassifier', AdaBoostClassifier()]]
          

hpspace_RandomForestClassifier = {
    'n_estimators': Integer(50, 500),
    'max_depth': Integer(5, 30),
    'min_samples_split': Integer(2, 10),
    'min_samples_leaf': Integer(1, 5),
    'max_features': Real(0.1, 0.5)
}


hpspace_KNeighborsClassifier = {
    'n_neighbors': Integer(3, 20),
    'weights': Categorical(['uniform', 'distance']),
    'algorithm': Categorical(['auto', 'kd_tree']),
    'leaf_size': Integer(10, 50),
    'p': Integer(1, 3),
}     

hpspace_LinearSVC = {
    'C': Real(1e-6, 1e+6, prior='log-uniform'),
    'tol': Real(1e-6, 1e-1, prior='log-uniform'),
    'class_weight': Categorical(['balanced', None])
}


hpspace_GradientBoostingClassifier = {
    'n_estimators': Integer(100, 1000),
    'max_depth': Integer(3, 10),
    'min_samples_split': Integer(2, 10),
    'min_samples_leaf': Integer(1, 10),
    'max_features': Categorical(['sqrt', 'log2']),
    'learning_rate': Real(0.001, 0.1, prior='log-uniform'),
}


hpspace_MLPClassifier = {
    'hidden_layer_sizes': Integer(10, 200, prior='log-uniform'),
    'activation': Categorical(['relu', 'logistic', 'tanh']),
    'solver': Categorical(['adam', 'sgd']),
    'alpha': Real(1e-6, 1e-2, prior='log-uniform'),
    'learning_rate': Categorical(['constant', 'invscaling', 'adaptive']),
    'learning_rate_init': Real(1e-4, 1e-2, prior='log-uniform'),
    'max_iter': Integer(100, 1000),
    'tol': Real(1e-6, 1e-2, prior='log-uniform')
}

hpspace_AdaBoostClassifier = {
    'n_estimators': Integer(10, 1000),
    'learning_rate': Real(0.001, 10, prior='log-uniform'),
    'algorithm': Categorical(['SAMME', 'SAMME.R']),
}

In [33]:

train_label_matrix = train_split['label'].to_numpy()
test_label_matrix = validation_split['label'].to_numpy()
print(train_label_matrix)

stratifier = StratifiedKFold(n_splits=2, shuffle=True, random_state=100)


counter=0

for model_name, model in models:
#  if counter != 5:
#    counter += 1
#    continue
#  else:
    counter += 1 
    print(model_name)
    params = globals()['hpspace_' + model_name]
    search = BayesSearchCV(model, params, n_iter=30, cv=stratifier,  verbose=5,
                           random_state=2, n_jobs=1, return_train_score=True)
    results = search.fit(X_vectorized, train_label_matrix)
    best_model = results.best_estimator_

    start = time.time()
    best_model.fit(X_vectorized, train_label_matrix)
    end = time.time()
    runtime = end - start
    model_config = results.cv_results_

    root = get_project_root()
    config = configparser.ConfigParser()
    config.read(root + '/src/modelling/my_config.ini')
    token_wandb = config['wandb']['TOKEN']
    wandb.login(key=token_wandb)

    model_params = model.get_params()
    model_params['model_type'] = type(model).__name__
    model_params['model_config'] = results.best_params_

    wandb.init(project='sports-article-classification-nordskog', tags=['sklearn', 'nordskog'], config=model_params)

    pipeline = make_pipeline(model)
    model = pipeline.fit(X_vectorized, train_label_matrix)

    pred_labels_tr = model.predict(X_vectorized)#.toarray()
    print(pred_labels_tr)
    pred_labels_te = model.predict(vectorizer.transform(validation_split['text']))#.toarray()


    score_tr = model.score(X_vectorized, train_label_matrix)
    #roc_auc_tr = roc_auc_score(train_label_matrix, pred_labels_tr, average='micro')
    loss_tr = categorical_crossentropy(train_label_matrix.astype(float), pred_labels_tr.astype(float))
    score_report_tr = classification_report(train_label_matrix, pred_labels_tr, target_names=label_encoder.classes_, output_dict=True)

    score_te = model.score(vectorizer.transform(validation_split['text']), test_label_matrix)
    #roc_auc_te = roc_auc_score(test_label_matrix, pred_labels_te, average='micro')
    loss_te = categorical_crossentropy(test_label_matrix.astype(float), pred_labels_te.astype(float))
    score_report_te = classification_report(test_label_matrix, pred_labels_te, target_names=label_encoder.classes_, output_dict=True)
    print(score_report_te)

    wandb.log({'eval/accuracy': score_te, 'eval/loss': loss_te,
              'eval/f1_macro': score_report_te['macro avg']['f1-score'], 'eval/f1_weighted': score_report_te['weighted avg']['f1-score'],
              'eval/score_report': score_report_te, 'train/train_runtime': runtime})
    wandb.finish()



[ 4  9  4 ... 11  9  6]
RandomForestClassifier
Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2] END max_depth=27, max_features=0.23480498788350176, min_samples_leaf=4, min_samples_split=6, n_estimators=426;, score=(train=0.771, test=0.652) total time=  28.4s
[CV 2/2] END max_depth=27, max_features=0.23480498788350176, min_samples_leaf=4, min_samples_split=6, n_estimators=426;, score=(train=0.790, test=0.623) total time=  28.7s
Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2] END max_depth=26, max_features=0.43525855474929764, min_samples_leaf=1, min_samples_split=6, n_estimators=270;, score=(train=0.860, test=0.649) total time=  31.8s
[CV 2/2] END max_depth=26, max_features=0.43525855474929764, min_samples_leaf=1, min_samples_split=6, n_estimators=270;, score=(train=0.884, test=0.615) total time=  30.5s
Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2] END max_depth=8, max_features=0.1798398123960324, min_samples_leaf=2, min_sampl

/usr/local/lib/python3.9/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2] END max_depth=30, max_features=0.1, min_samples_leaf=1, min_samples_split=2, n_estimators=500;, score=(train=0.939, test=0.658) total time=  27.9s
[CV 2/2] END max_depth=30, max_features=0.1, min_samples_leaf=1, min_samples_split=2, n_estimators=500;, score=(train=0.938, test=0.635) total time=  27.9s
Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2] END max_depth=24, max_features=0.1027819616289603, min_samples_leaf=5, min_samples_split=2, n_estimators=77;, score=(train=0.733, test=0.642) total time=   3.1s
[CV 2/2] END max_depth=24, max_features=0.1027819616289603, min_samples_leaf=5, min_samples_split=2, n_estimators=77;, score=(train=0.745, test=0.629) total time=   2.5s
Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2] END max_depth=21, max_features=0.5, min_samples_leaf=2, min_samples_split=5, n_estimators=50;, score=(train=0.759, test=0.635) total time=   4.5s
[CV 2/2] END max

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


[ 4  9  4 ... 11  9  6]
{'Chances': {'precision': 0.8888888888888888, 'recall': 0.13333333333333333, 'f1-score': 0.2318840579710145, 'support': 60}, 'Club Detail': {'precision': 0.782608695652174, 'recall': 0.2857142857142857, 'f1-score': 0.4186046511627907, 'support': 63}, 'Club Drama': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1}, 'Goal/Assist': {'precision': 0.6272727272727273, 'recall': 0.9241071428571429, 'f1-score': 0.7472924187725631, 'support': 224}, 'Ignore': {'precision': 0.8356164383561644, 'recall': 0.9172932330827067, 'f1-score': 0.8745519713261649, 'support': 133}, 'Injuries': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 12}, 'Irrelevant': {'precision': 0.43859649122807015, 'recall': 0.46296296296296297, 'f1-score': 0.45045045045045046, 'support': 162}, 'Personal Drama': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1}, 'Player Detail': {'precision': 0.6363636363636364, 'recall': 0.10294117647058823, 'f1-score': 0.17721

...

AdaBoostClassifier
Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2] END algorithm=SAMME.R, learning_rate=0.02228691100328716, n_estimators=786;, score=(train=0.364, test=0.343) total time=  26.0s
[CV 2/2] END algorithm=SAMME.R, learning_rate=0.02228691100328716, n_estimators=786;, score=(train=0.374, test=0.345) total time=  24.0s
Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2] END algorithm=SAMME.R, learning_rate=2.2520889904053125, n_estimators=101;, score=(train=0.084, test=0.071) total time=   3.4s
[CV 2/2] END algorithm=SAMME.R, learning_rate=2.2520889904053125, n_estimators=101;, score=(train=0.011, test=0.009) total time=   2.7s
Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2] END algorithm=SAMME, learning_rate=0.006286343728695145, n_estimators=238;, score=(train=0.353, test=0.343) total time=   6.3s
[CV 2/2] END algorithm=SAMME, learning_rate=0.006286343728695145, n_estimators=238;, score=(train=0.346, test=0.352) total

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


[ 4 11  3 ...  3  3  3]
{'Chances': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 60}, 'Club Detail': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 63}, 'Club Drama': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1}, 'Goal/Assist': {'precision': 0.23956043956043957, 'recall': 0.9732142857142857, 'f1-score': 0.3844797178130512, 'support': 224}, 'Ignore': {'precision': 0.935064935064935, 'recall': 0.5413533834586466, 'f1-score': 0.6857142857142857, 'support': 133}, 'Injuries': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 12}, 'Irrelevant': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 162}, 'Personal Drama': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1}, 'Player Detail': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 68}, 'Quote': {'precision': 0.5, 'recall': 0.020512820512820513, 'f1-score': 0.03940886699507389, 'support': 195}, 'Rodt/Gult kort': {'precision': 0.45, 'recall

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classificat

eval/accuracy,▁
eval/f1_macro,▁
eval/f1_weighted,▁
eval/loss,▁
train/train_runtime,▁
eval/accuracy,0.30108
eval/f1_macro,0.16119
eval/f1_weighted,0.20866
eval/loss,47073.34814
train/train_runtime,20.77577


In [34]:
sofus_alert()

Sofus has sent an alert - Check slack!
